In [1]:
from LOTlib3.Grammar import Grammar
from math import log
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.DataAndObjects import FunctionData
from LOTlib3.TopN import TopN
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler
from LOTlib3.Eval import primitive

In [3]:
# intializing basic hypothesis, will edit later
class ARCHypothesis(LOTHypothesis):
    def __init__(self, **kwargs):
        LOTHypothesis.__init__(self, grammar=grammar, display="lambda grid: %s", **kwargs)
        
    def __call__(self, *args):
        try:
            # try to do it from the superclass
            return LOTHypothesis.__call__(self, *args)
        except ZeroDivisionError:
            # and if we get an error, return nan
            return float("nan")

    def compute_single_likelihood(self, datum):
        # for each correct pixel, add log(1), for each incorrect, add log(exp(-b))
        ll = 0
        input_grid = self(*datum.input)
        output_grid = datum.output
        # assuming input and output are the same size for now
        height, width = len(input_grid[:,0]), len(input_grid[0,:])
        for i in range(height):
            for j in range(width):
                if input_grid[i][j] == output_grid[i][j]:
                    ll += log(1)
#                 elif input_grid[i][j] == 0 and output_grid[i][j] != 0:
#                     ll += log(0.1)
                else:
                    ll += log(0.05)
        # print(self.value, ll)
        return ll
        
    def display_hypothesis(self):
        return self.value

In [4]:
# initializing grammar
grammar = Grammar(start='GRID')

# Boolean
grammar.add_rule('BOOLEAN', True, None, 1.0)
grammar.add_rule('BOOLEAN', False, None, 1.0)
grammar.add_rule('BOOLEAN', 'not %s', ['BOOLEAN'], 1.0)
grammar.add_rule('BOOLEAN', '%s or %s', ['BOOLEAN', 'BOOLEAN'], 1.0)
grammar.add_rule('BOOLEAN', '%s and %s', ['BOOLEAN', 'BOOLEAN'], 1.0)
grammar.add_rule('BOOLEAN', 'equals_object', ['OBJECT', 'OBJECT'], 1.0)
grammar.add_rule('BOOLEAN', 'equals_location', ['LOC', 'LOC'], 1.0)
grammar.add_rule('BOOLEAN', 'equals_color', ['COLOR', 'COLOR'], 1.0)
grammar.add_rule('BOOLEAN', 'equals_int', ['INT', 'INT'], 1.0)
grammar.add_rule('BOOLEAN', '%s > %s', ['INT', 'INT'], 1.0)
grammar.add_rule('BOOLEAN', '%s < %s', ['INT', 'INT'], 1.0)
grammar.add_rule('BOOLEAN', 'is_touching', ['OBJECT', 'OBJECT'], 1.0)
grammar.add_rule('BOOLEAN', 'has_color', ['OBJECT', 'COLOR'], 1.0)
grammar.add_rule('BOOLEAN', 'is_color', ['LOC', 'COLOR'], 1.0)
grammar.add_rule('BOOLEAN', 'is_size', ['OBJECT', 'INT'], 1.0)

# Integer
for n in range(10):
    grammar.add_rule('INT', n, None, 1.0)
grammar.add_rule('INT', 'object_size', ['OBJECT'], 1.0)
grammar.add_rule('INT', 'num_objects', ['GRID'], 1.0)
grammar.add_rule('INT', 'num_grid_colors', ['GRID'], 1.0)
grammar.add_rule('INT', 'num_colors', ['OBJECT'], 1.0)
grammar.add_rule('INT', 'grid_height', ['GRID'], 1.0)
grammar.add_rule('INT', 'grid_width', ['GRID'], 1.0)


# Colors
for n in range(10):
    grammar.add_rule('COLOR', str(n), None, 1.0)
grammar.add_rule('COLOR', 'object_color', ['OBJECT'], 1.0)
grammar.add_rule('COLOR', 'background_color', ['GRID'], 1.0)
grammar.add_rule('COLOR', 'get_color', ['LOC'], 1.0)

# Location
for i in range(30):
    for j in range(30):
        grammar.add_rule('LOC', (i,j), None, 1.0)
grammar.add_rule('LOC', 'top_left', ['OBJECT'],1.0)
grammar.add_rule('LOC', 'top_right', ['OBJECT'],1.0)
grammar.add_rule('LOC', 'bottom_left', ['OBJECT'],1.0)
grammar.add_rule('LOC', 'bottom_right', ['OBJECT'],1.0)
grammar.add_rule('LOC', 'top_left_corner', ['GRID'],1.0)
grammar.add_rule('LOC', 'top_right_corner', ['GRID'],1.0)
grammar.add_rule('LOC', 'bottom_left_corner', ['GRID'],1.0)
grammar.add_rule('LOC', 'bottom_right_corner', ['GRID'],1.0)

# Dictionary

# List/Set

# SetB
# SetC
# AllColors(Grid)
# SetL
# SetO

# Objects
grammar.add_rule('OBJECT', 'object_by_color', ['GRID', 'COLOR'], 1.0)


# Grid
grammar.add_rule('GRID', 'grid', None, 1.0)
grammar.add_rule('GRID', 'reflection_horizontal', ['GRID'], 1.0)
grammar.add_rule('GRID', 'reflection_vertical', ['GRID'], 1.0)
grammar.add_rule('GRID', 'rotation_90', ['GRID'], 1.0)
grammar.add_rule('GRID', 'rotation_180', ['GRID'], 1.0)
grammar.add_rule('GRID', 'rotation_270', ['GRID'], 1.0)
grammar.add_rule('GRID', 'scale', ['GRID'], 1.0)
grammar.add_rule('GRID', 'repeat', ['GRID'], 1.0)
grammar.add_rule('GRID', 'gravity', ['GRID'], 1.0)
grammar.add_rule('GRID', 'replace', ['OBJECT','OBJECT'], 1.0)
grammar.add_rule('GRID', 'AND', ['GRID','GRID'], 1.0)
grammar.add_rule('GRID', 'OR', ['GRID','GRID'], 1.0)
grammar.add_rule('GRID', 'XOR', ['GRID','GRID'], 1.0)


GRID -> XOR['GRID', 'GRID']	w/ p=1.0

In [15]:
# Object primitives
@primitive
def object_by_color(grid, color):
    obj = []
    coords = list(zip(np.where(grid==color)[0],np.where(grid==color)[1]))
    for coord in coords:
        obj.append([coord[0],coord[1],color])
    return obj

# Boolean primitives
@primitive
def equals_object(obj1,obj2):
    return sorted(relative_positions(obj1)) == sorted(relative_positions(obj2))
        
@primitive
def equals_location(loc1,loc2):
    if loc1[0] == loc2[0] and loc1[1] == loc2[1]:
        return True
    return False
    
@primitive
def equals_color(col1,col2):
    if col1 == col2:
        return True
    return False

@primitive
def equals_int(int1,int2):
    if int1==int2:
        return True
    return False


# Location primitives
@primitive
def top_left(obj):
    sorted_obj = sorted(obj, key=lambda x: x[0])
    return sorted_obj[0]

# Helper functions
def relative_positions(obj):
    dx, dy = top_left(obj)[0], top_left(obj)[1]
    rel_obj = []
    for coord in obj:
        rel_obj.append([coord[0]-dx,coord[1]-dy, coord[2]])
    return rel_obj